In [ ]:
import twarc
import os
import pandas as pd

In [ ]:
s="""@ICRC_Nairobi
@ICRC_library
@RedCrossLebanon
@RedCross
@BritishRedCross
@RedCrossAU
@irishredcross
@philredcross
@IFRC_Europe
@NepalRedCross"""

In [ ]:
fscript=open("download/download.sh",'w')
for ind,org in enumerate(s.split('\n')):
    fngo=open("download/ngos{0}.txt".format(ind),'w')
    fngo.write(org.replace("@",""))
    fngo.close()
    fscript.write("twarc2 timelines --use-search download/ngos{0}.txt download{1}.jsonl\n".format(ind,org.replace("@","")))
fscript.close()

In [ ]:
# %sh download.sh

In [ ]:
fscript=open("dehydrate/dehydrate.sh",'w')
for file in os.listdir('download'):
    if file.split('.')[1]=='jsonl':
        fscript.write("twarc2 dehydrate download/{0} dehydrate/{1}.txt\n".format(file,file.split('.')[0]))
        # fscript.write("twarc2 conversations --archive {0}.txt > {1} \n".format(file.split('.')[0]))
fscript.close()

In [ ]:
# %sh dehydrate.sh

In [ ]:
fscript=open("hydrate/hydrate.sh",'w')
for file in os.listdir('dehydrate'):
    if file.split('.')[1]=='txt':
        fscript.write("twarc2 hydrate dehydrate/{0} hydrate/{1}.json\n".format(file,file.split('.')[0]))
        # fscript.write("twarc2 conversations --archive {0}.txt > {1} \n".format(file.split('.')[0]))
fscript.close()

In [ ]:
# %sh hydrate.sh

In [ ]:
for file in os.listdir('hydrate'):
    if file.split('.')[1]=='json':
        df=pd.read_json('hydrate/'+file,lines=True)
        conversation_ids=[]
        for line in df['data']:
            for element in line:
                if element['id']==element['conversation_id']:
                    conversation_ids.append(element['id'])
        print(len(set(conversation_ids)))
        with open("hydrate/{0}.txt".format(file.split('.')[0]), 'w') as fp:
            for id in set(conversation_ids):
                fp.write("%s\n" % id)

In [ ]:
fscript=open("conversations/conversations.sh",'w')
for file in os.listdir('hydrate'):
    if file.split('.')[1]=='txt':
        fscript.write("twarc2 conversations --archive hydrate/{0} conversations/{1}.json\n".format(file,file.split('.')[0]))
fscript.close()

In [ ]:
# %sh conversations.sh

Code to extend the conversation_ids collected not only from "data", but also from "includes". This way we get independent tweets of users mentioninig ICRC, on which they got replies:

In [ ]:
for file in os.listdir('hydrate'):
    if file.split('.')[1]=='json':
        df=pd.read_json('hydrate/'+file,lines=True)
        conversation_ids=[]
        for line in df['includes']:
            for element in line["tweets"]:
                conversation_ids.append(element['conversation_id'])
        print(len(set(conversation_ids)))
        with open("hydrate/Extended_{0}.txt".format(file.split('.')[0]), 'w') as fp:
            for id in set(conversation_ids):
                fp.write("%s\n" % id)

In [ ]:
NGO_list = ["Australia_ids", "Britain_ids", "Canada_ids", "Ireland_ids", "Nairobi_ids", "nepal_ids", "RedCrossLebanon", "RedCross", "philredcross", "IFRC_Europe", "ICRC_library"]

In [ ]:
for ngo in NGO_list:
    df_initial = pd.read_csv("hydrate/{file}.txt".format(file=ngo), delimiter="\t", header = None)
    df_extended = pd.read_csv("hydrate/Extended_{file}.txt".format(file=ngo), delimiter="\t", header = None)
    new_ids = pd.concat([df_extended, df_initial, df_initial]).drop_duplicates(keep=False)
    new_ids.to_csv("conversations_extended/New_{file}.txt".format(file=ngo), header=None, index=None)
    print(len(new_ids))

In [ ]:
fscript=open("conversations_extended/conversations_extended.sh",'w')
for file in os.listdir('conversations_extended'):
    if file.split('.')[1]=='txt':
        fscript.write("twarc2 conversations --archive conversations_extended/{0} conversations_extended/{1}.json\n".format(file,file.split('.')[0]))
fscript.close()